# **1. Load Necessary Library**

In [1]:
import librosa
import tensorflow as tf
import numpy as np

SAVED_MODEL_PATH = "./keyword_model_1.h5"
SAMPLES_TO_CONSIDER = 22050

# **2. Build Keyword spotting Function**

In [2]:
class _Keyword_Spotting_Service:

    model = None
    _mapping = [
        "down",
        "go",
        "stop",
        "up"
            ]
    _instance = None


    def predict(self, file_path):

        # extract MFCC
        MFCCs = self.preprocess(file_path)

        # we need a 4-dim array to feed to the model for prediction: (# samples, # time steps, # coefficients, 1)
        MFCCs = MFCCs[np.newaxis, ..., np.newaxis]

        # get the predicted label
        predictions = self.model.predict(MFCCs)
        predicted_index = np.argmax(predictions)
        predicted_keyword = self._mapping[predicted_index]
        return predicted_keyword


    def preprocess(self, file_path, num_mfcc=13, n_fft=2048, hop_length=512):
        # load audio file
        signal, sample_rate = librosa.load(file_path)

        if len(signal) >= SAMPLES_TO_CONSIDER:
            # ensure consistency of the length of the signal
            signal = signal[:SAMPLES_TO_CONSIDER]

            # extract MFCCs
            MFCCs = librosa.feature.mfcc(y=signal, sr=sample_rate, n_mfcc=num_mfcc, n_fft=n_fft,
                                         hop_length=hop_length)
        return MFCCs.T



In [3]:
def Keyword_Spotting_Service():

    # ensure an instance is created only the first time the factory function is called
    if _Keyword_Spotting_Service._instance is None:
        _Keyword_Spotting_Service._instance = _Keyword_Spotting_Service()
        _Keyword_Spotting_Service.model = tf.keras.models.load_model(SAVED_MODEL_PATH)
    return _Keyword_Spotting_Service._instance

# **3.Prediction**

In [4]:
# create 2 instances of the keyword spotting service
kss = Keyword_Spotting_Service()
kss1 = Keyword_Spotting_Service()
# check that different instances of the keyword spotting service point back to the same object (singleton)
assert kss is kss1

In [6]:
import pandas
data=pandas.read_csv('key_audio_2.csv')
data

,file_name,key
0,./inputs/Voice 001.wav,up
1,./inputs/Voice 002.wav,down
2,./inputs/Voice 003.wav,up
3,./inputs/Voice 004.wav,up
4,./inputs/Voice 005.wav,stop
5,./inputs/Voice 006.wav,go
6,./inputs/Voice 007.wav,stop
7,./inputs/Voice 008.wav,stop
8,./inputs/Voice 009.wav,go
9,./inputs/Voice 010.wav,down


In [7]:
prediction=[]
def travis(row):
    keyword = kss.predict(row.file_name)
    prediction.append(keyword)

out=data.apply(lambda row: travis(row),axis=1)

data['prediction']=prediction
data

1/1 [==============================] - 0s 9ms/step


,file_name,key,prediction
0,./inputs/Voice 001.wav,up,go
1,./inputs/Voice 002.wav,down,down
2,./inputs/Voice 003.wav,up,go
3,./inputs/Voice 004.wav,up,go
4,./inputs/Voice 005.wav,stop,stop
5,./inputs/Voice 006.wav,go,up
6,./inputs/Voice 007.wav,stop,stop
7,./inputs/Voice 008.wav,stop,down
8,./inputs/Voice 009.wav,go,go
9,./inputs/Voice 010.wav,down,down


In [8]:
def sentence_accuracy(text1, text2):
    return text1==text2

# Calculate accuracy for each row
accuracies = []
for text1, text2 in zip(data['key'], data['prediction']):
    text2=text2.upper()
    text1=text1.upper()
    accuracy = sentence_accuracy(text1, text2)
    accuracies.append(accuracy)

# Add the accuracies as a new column in the DataFrame
data['sentence_level_accuracy'] = accuracies

# Calculate the overall average accuracy
average_accuracy = data['sentence_level_accuracy'].mean()

# Print the results
print(f"Average Accuracy: {average_accuracy:.2%}")

Average Accuracy: 50.00%


In [9]:
data

,file_name,key,prediction,sentence_level_accuracy
0,./inputs/Voice 001.wav,up,go,False
1,./inputs/Voice 002.wav,down,down,True
2,./inputs/Voice 003.wav,up,go,False
3,./inputs/Voice 004.wav,up,go,False
4,./inputs/Voice 005.wav,stop,stop,True
5,./inputs/Voice 006.wav,go,up,False
6,./inputs/Voice 007.wav,stop,stop,True
7,./inputs/Voice 008.wav,stop,down,False
8,./inputs/Voice 009.wav,go,go,True
9,./inputs/Voice 010.wav,down,down,True
